In [60]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import ensemble, metrics

In [2]:
# читаем массив данных
data = pd.read_csv('lenta_example.csv')
print(len(data))
data.head()

24467


,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
1,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина
2,https://lenta.ru/news/2018/12/15/ovechkin/,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт,Хоккей
3,https://lenta.ru/news/2018/12/15/newyear/,Названы регионы России с самым дорогим и дешев...,Производитель онлайн-касс «Эвотор» проанализир...,Экономика,Деньги
4,https://lenta.ru/news/2018/12/15/gaz/,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика,Госэкономика


**Векторизация** - процесс превращения объекта в массив чисел

До:

$$ a = \text{"мама мыла раму"}, $$
$$ b = \text{"папа мыл апельсин"} $$

После:

|   | апельсин  |  мама | мыть  |  папа |  рама  |
|---|---|---|---|---|---|
|a |  0 | 1  | 1  |  0 | 1  |
| b |  1 | 0  | 1  |  1 | 0  |

In [47]:
'Медведев' in vectorizer.get_feature_names()

False

In [48]:
# делаем векторизацию - превращаем текст в набор чисел
vectorizer=TfidfVectorizer(analyzer='word')
vectorizer.fit_transform(data['text'] )

<24467x195536 sparse matrix of type '<class 'numpy.float64'>'
	with 3151943 stored elements in Compressed Sparse Row format>

## Алгоритм действий
Дано: data + answers

- Делим данные: 

$$data \rightarrow train\_data + test\_data$$


- Обучаем: 

$$(train\_data + answers) \xrightarrow{\text{обучение}}  model.fit() \rightarrow \textit{обученная модель} $$


- Проверяем:

$$test\_data \xrightarrow{\text{предсказание}} model.predict()  \rightarrow \textit{ответы модели}$$


- Оцениваем результат:

$$Accuracy = \frac{Угаданные\ верно\ ответы}{Всего\ данных}$$

In [76]:
# формируем обучающую выборку
train_data = data.tail(10000)['text']
X_train = vectorizer.fit_transform(train_data).toarray()
y_train = data.tail(10000)['topic']

# формируем тестовую выборку
test_data = data.head(10000)['text']
X_test = vectorizer.transform(test_data).toarray()
y_test = data.head(10000)['topic']

In [50]:
y_train

14467          Экономика
14468           Ценности
14469              Спорт
14470           Культура
14471                Дом
              ...       
24462          Экономика
24463           Ценности
24464           Ценности
24465    Наука и техника
24466              Спорт
Name: topic, Length: 10000, dtype: object

In [77]:
%%time
# обучаем модель
# model = LogisticRegression(solver='liblinear')
model = ComplementNB()
model.fit(X_train, y_train)
result = model.predict(X_test)

Wall time: 11.2 s


In [62]:
result

array(['Спорт', 'Бывший СССР', 'Спорт', ..., 'Экономика',
       'Наука и техника', 'Культура'], dtype='<U15')

In [63]:
# посмотрим, как распределены классы
data['topic'].value_counts()

Спорт              5458
Экономика          4915
Наука и техника    3511
Бывший СССР        3260
Культура           3171
Дом                2088
Ценности           2064
Name: topic, dtype: int64

In [79]:
%%time
# посмотрим на результаты
print ('Доля правильных ответов:')
print ('train - ', model.score(X_train, y_train))
print ('test - ', model.score(X_test, y_test))

Доля правильных ответов:
train -  0.9733
test -  0.9272
Wall time: 7.27 s


In [80]:
pd.DataFrame(metrics.confusion_matrix(y_test, result), index=model.classes_, columns=model.classes_)

,Бывший СССР,Дом,Культура,Наука и техника,Спорт,Ценности,Экономика
Бывший СССР,1294,10,9,33,19,3,121
Дом,0,590,11,4,12,1,122
Культура,33,3,1201,4,39,5,22
Наука и техника,10,0,7,1314,5,3,48
Спорт,4,1,1,0,2206,0,4
Ценности,3,4,45,23,24,669,16
Экономика,12,22,6,10,16,13,1998


In [81]:
# попробуем проверить модель вручную
model.predict(vectorizer.transform(['ученые открыли лекарство от рака']))

array(['Наука и техника'], dtype='<U15')

In [25]:
# аренда жилья превысила спрос 2018 года
# на форуме инноваций Греф презентовал новую платформу для игры на бирже
# Трамп наложил новые тарифы на сталь из китая из-за этого было открыто несколько заводов

In [82]:
model.predict(vectorizer.transform(['Названы регионы России с самым дорогим и дешевым']))

array(['Дом'], dtype='<U15')

In [83]:
model.predict(vectorizer.transform(['выставка']).toarray())

array(['Культура'], dtype='<U15')